### Read training, dev and unlabeled test data

The following provides a starting code (Python 3) of how to read the labeled training and dev cipher text, and unlabeled test cipher text, into lists.

In [103]:
train, dev, test = [], [], []

In [104]:
for x in open('./train_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    train.append(x)
print (len(train))
print (train[:3])

16220
[[0, 'lkêcê yoúc cêêö y#êjl lw mówám Újám j Úêê# ütlk Úol lkêú z#ê ctöé8ú ówl xoóóú éê#xw#öê#c .'], [0, '6êcétlê jolêot8 zc éê#xw#öjóáê , tl zc j #jlkê# 8tcl8êcc jöÚ8ê 6wüó lkê öt668ê wx lkê #wj6 , ükê#ê lkê lkêöjltá t#wótêc j#ê lww wÚ2twoc jó6 lkê cê+oj8 éw8tltác lww cöoy .'], [0, 'tx lktc kw8t6jú öw2tê tc coééwcê6 lw Úê j ytxl , cwöêÚw6ú oóü#jééê6 tl êj#8ú , lwwm wol j88 lkê yww6 cloxx , jó6 8êxl Úêktó6 lkê á#jé ( 8tlê#j88ú ) .']]


In [105]:
for x in open('./dev_enc.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r').split('\t')
    # x[0] will be the label (0 or 1), and x[1] will be the ciphertext sentence.
    x[0] = int(x[0]) 
    dev.append(x)
print (len(dev))
print (dev[:3])

2027
[[1, 'ów8jó Ú#j2ê8ú l#êj6c ükê#ê xêü jöê#tájó xt8öc 6j#ê lw 6ê82ê 77 tólw lkê üw#86 wx jöÚt2j8êóáê jó6 jöÚtyotlú <<<'], [0, 'ê2êó öo#ékú zc ê+éê#l áwötá ltötóy jó6 xjöê6 ákj#tcöj áj ózl #êcáoê lktc êxxw#l .'], [1, 'üt88 jcco#ê68ú #jóm jc wóê wx lkê á8ê2ê#êcl , öwcl 6êáêélt2ê8ú jöoctóy áwöê6têc wx lkê úêj# .']]


#### Different from 'train' and 'dev' that are both list of tuples, 'test' will be just a list.

In [106]:
for x in open('./test_enc_unlabeled.tsv', encoding='utf-8'):
    x = x.rstrip('\n\r')
    test.append(x)
print (len(test))
print (test[:3])

2028
['j 6t6jáltá jó6 6o88 6wáoöêólj#ú y8w#txútóy cwxlüj#ê jój#ákú .', 'ówlktóy cltámc , #êj88ú , ê+áêél j 8tóyê#tóy á#êêétóêcc wóê xêê8c x#wö Úêtóy 6#jyyê6 lk#woyk j cj6 , cw#6t6 oót2ê#cê wx yoóc , 6#oyc , j2j#táê jó6 6jöjyê6 6#êjöc .', 'öo#ékú jó6 üt8cwó jáloj88ú öjmê j é#êllú yww6 lêjö <<< Úol lkê é#wvêál co##woó6tóy lkêö tc 6tcl#êcctóy8ú #wlê .']


#### You can split every sentence into lists of words by white spaces.

In [107]:
train_split = [[x[0], x[1].split(' ')] for x in train]
dev_split = [[x[0], x[1].split(' ')] for x in dev]
test_split = [[x.split(' ')] for x in test]

### Main Code Body

You may choose to experiment with different methods using your program. However, you need to embed the training and inference processes at here. We will use your prediction on the unlabeled test data to grade, while checking this part to understand how your method has produced the predictions.

In [108]:
# Eventually, results need to be a list of 2028 0 or 1's
results = []

## Train fasttext classification model

In [119]:
import fasttext
import numpy as np

def write_data_ft(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        for example in data:
            print(f'{example[1]}\t__label__{example[0]}', file=f)

def accuracy(results, labels):
    return np.sum(results == labels) / len(labels)

def get_predictions(model, X):
    predictions = model.predict(X)
    predictions = list(map(lambda x: int(x[0][-1]), predictions[0]))
    return np.array(predictions)

write_data_ft('train_enc_ft.tsv', train)
write_data_ft('dev_enc_ft.tsv', dev)

model = fasttext.train_supervised(
    'train_enc_ft.tsv', autotuneValidationFile='dev_enc_ft.tsv', verbose=1, dim=96)

dev = np.array(dev, dtype=object)
predictions = get_predictions(model, list(dev[:, 1]))

labels = np.array(dev[:, 0])

print(f'Dev accuracy: {accuracy(predictions, labels)}')

Progress: 100.0% Trials:   73 Best score:  0.904292 ETA:   0h 0m 0s
Training again with best arguments
Read 0M words
Number of words:  20861
Number of labels: 2


Dev accuracy: 0.9028120374938332


Progress: 100.0% words/sec/thread:  377780 lr:  0.000000 avg.loss:  0.097461 ETA:   0h 0m 0s


In [120]:
print(f'model.lr: {model.lr}')
print(f'model.epoch {model.epoch}')
print(f'model.wordNgrams {model.wordNgrams}')
print(f'model.minn {model.minn}')
print(f'model.maxn {model.maxn}')
print(f'Word embeddign size: {model.dim}')

model.lr: 0.09124087041869901
model.epoch 100
model.wordNgrams 5
model.minn 2
model.maxn 5
Word embeddign size: 96


In [121]:
results = get_predictions(model, test)

### Output Prediction Result File

You will need to submit a prediction result file. It should have 2028 lines, every line should be either 0 or 1, which is your model's prediction on the respective test set instance.

In [122]:
# suppose you had your model's predictions on the 2028 test cases read from test_enc_unlabeled.tsv, and 
#those results are in the list called 'results'
assert (len(results) == 2028)

In [123]:
# make sure the results are not float numbers, but intergers 0 and 1
results = [int(x) for x in results]

In [124]:
# write your prediction results to 'upload_predictions.txt' and upload that later
with open('upload_predictions.txt', 'w', encoding = 'utf-8') as fp:
    for x in results:
        fp.write(str(x) + '\n')